<a href="https://colab.research.google.com/github/NShravanReddy/DeepLearning/blob/main/Qwen_2_5_Conversational_%2B_Unsloth_2x_faster_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

Features in the notebook:
1. Uses Maxime Labonne's [FineTome 100K](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset.
1. Convert ShareGPT to HuggingFace format via `standardize_sharegpt`
2. Train on Completions / Assistant only via `train_on_responses_only`
3. Unsloth now supports Torch 2.4, all TRL & Xformers versions & Python 3.12!

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.0: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.0 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Qwen-2.5` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Qwen2.5 renders multi turn conversations like below:

```
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there! How are you?<|im_end|>
<|im_start|>user
I'm great thanks!<|im_end|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("hackercupai/hackercup", split = "train")

Generating sample split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating full split:   0%|          | 0/284 [00:00<?, ? examples/s]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

We look at how the conversations are structured for item 5:

In [ ]:
dataset[5]["conversations"]

And we see how the chat template transformed these conversations.

**[Notice]** Qwen 2.5 Instruct's default chat template default adds `"You are Qwen, created by Alibaba Cloud. You are a helpful assistant."`, so do not be alarmed!

In [ ]:
dataset[5]["text"]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content":
     """ Give me code in c language.
    As the queen of an ant colony, it’s your job to ensure that the *ant*ire colony works together. Your colony has \(N\) worker ants, the \(i\)th of which is curr*ant*ly at coordinates \((X_i, Y_i)\). To align the efforts of all of your worker ants, you would like them to all be on the same line on the plane. How many of your ants need to move to get them to all lie on the same line?

As is frequ*ant*ly the case in managem*ant*, you don’t need an exact answer, but you do need some degree of accuracy. If the true minimum number of ants that need to move is \(M\), then any answer between \(M\) and \(2*M\) (inclusive) will be accepted.

# Constraints
\(1 \leq T \leq 75\)
\(2 \leq N \leq 1{,}000{,}000\)
\(0 \leq |X_i|, |Y_i| \leq 1{,}000{,}000{,}000\)

In each test case, no two ants will be at the same position.

The sum of \(N\) across all test cases is at most \(4{,}000{,}000\).

# Input Format
Input begins with an integer \(T\), the number of test cases. Each case starts with a line that contains the integer \(N\). Then \(N\) lines follow, the \(i\)th of which contains the integers \(X_i\) and \(Y_i\).
114
4 17
1
2
5
10
4 4
1
2
5
10
2 22
22
22
3 1000000000
1000000000
1000000000
1000000000
1 10
12
1 100
12
1 10
10
1 10
9
1 10
11
2 10
10
5
2 10
10
15
5 100
15
14
20
28
16
5 100
16
15
20
28
17
5 987654321
345678912
456789123
567891234
678912345
789123456
989 1975
718
723
397
156
565
531
792
45
377
612
191
8
891
328
121
112
966
515
466
879
437
600
610
775
665
844
863
373
58
521
969
381
56
846
388
209
478
495
85
284
555
945
23
645
853
652
652
962
451
332
217
518
30
566
903
659
970
660
965
825
37
738
206
274
83
310
93
457
387
283
4
342
25
130
924
595
739
388
315
886
293
208
964
180
755
572
822
829
627
93
546
473
119
475
53
368
673
390
449
41
332
279
639
825
6
255
598
612
223
626
965
323
553
205
122
29
793
429
12
551
972
997
541
88
899
506
173
668
750
565
524
109
818
933
91
830
233
420
599
503
900
700
985
950
447
423
131
102
363
262
158
916
630
941
873
224
277
324
545
324
710
742
875
225
960
421
796
730
295
769
75
919
698
555
875
247
713
740
693
438
627
185
750
415
669
919
730
103
710
542
802
925
720
115
794
398
289
195
501
356
975
90
621
14
893
857
58
70
817
904
437
690
466
455
701
82
169
694
760
689
682
80
97
338
910
598
566
817
384
572
225
599
405
828
106
447
618
179
996
354
479
717
77
625
234
928
68
270
262
133
768
459
320
10
478
958
876
819
441
930
924
337
249
40
926
725
144
159
687
40
652
179
560
581
927
17
971
223
90
979
528
190
44
760
433
604
739
613
924
240
66
792
504
63
696
80
622
526
402
278
391
772
95
908
702
730
440
250
457
454
317
154
110
893
57
312
25
785
86
939
908
640
624
30
45
797
317
856
268
872
166
103
254
625
764
402
805
388
83
175
271
775
25
335
400
264
637
702
972
22
896
88
961
808
752
959
140
434
929
515
790
613
543
131
799
991
893
790
762
291
281
559
977
529
780
138
576
906
650
610
554
670
495
881
257
550
662
169
802
978
974
370
627
828
951
941
436
640
824
823
787
176
678
40
479
67
795
871
610
862
462
312
865
601
207
525
918
601
816
77
18
159
755
776
839
837
411
433
876
999
952
945
563
628
320
259
800
151
66
466
690
881
894
607
266
953
326
814
550
197
259
727
297
744
977
670
243
945
771
826
865
37
138
555
977
179
288
785
699
35
123
579
808
296
394
357
249
245
586
95
512
230
116
525
163
292
353
17
519
23
804
801
695
792
986
816
398
17
289
947
307
146
833
332
910
506
273
110
217
261
263
160
905
45
560
631
319
583
340
188
975
820
645
431
905
347
374
337
540
998
944
437
337
696
318
436
806
244
564
89
245
173
533
545
492
716
769
193
94
671
770
546
172
242
219
439
42
566
791
304
151
816
627
77
264
220
836
851
459
805
555
384
151
38
303
709
576
91
898
188
387
928
639
394
808
953
605
591
769
443
956
647
571
308
561
589
854
869
523
719
940
682
216
359
210
48
858
542
848
444
806
264
604
613
932
519
817
373
675
325
2
805
522
434
828
256
188
303
44
923
997
40
571
720
418
600
494
917
271
637
322
363
170
78
692
495
310
570
527
701
950
664
547
150
190
512
326
721
952
323
35
218
195
249
328
962
651
21
788
146
204
765
505
182
838
16
845
509
393
315
689
40
293
779
959
163
377
644
940
573
731
929
181
58
818
110
29
296
382
99
542
410
463
960
266
430
817
554
63
834
243
993
294
862
839
118
398
203
264
441
96
302
178
281
940
619
704
335
157
510
609
994
237
279
404
922
334
516
69
500
910
876
104
149
126
839
755
732
577
194
673
253
418
8
118
936
921
558
531
689
930
308
79
356
539
620
50
354
219
827
976
363
575
152
341
504
666
584
935
535
626
786
768
686
946
662
516
222
97
972
900
945
298
696
969
879
967
321
643
609
200
825
445
611
147
5
255
532
181
771
249
524
282
873
178
102
198
146
157
465
573
267
426
632
302
88
392
134
95
863
797
403
294
79
970
850
6
102
484
608
397
581
606
124
541
284
620
138
934
675
135
742
533
83
784
603
956
742
211
93
384
339
905
117
456
920
472
92
352
80
708
244
992
539
409
779
4
535
534
100
60
862
753
801
138
453
650
997
908
233
951
841
874
468
145
467
876
14
971
691
926
805
971
968
653
813
87
795
461
462
781
937
664
342
425
624
603
546
1
730
202
103
911
559
193
350
50
782
234
941
158
519
36
914
288
839
201
620
57
448
493
903
241
361
170
996
655
909
985
663
454
788
387
944
202
46
252
897
967
449
478
462
389
71
668
846
5
306
186
940
195
602
897
859
417
142
111
607
910
604
107
398
551
215
858
578
417
463
469
984 1964
872
138
506
713
990
980
429
738
130
670
928
99
473
388
108
278
813
805
246
262
489
319
884
502
884
134
64
100
434
354
165
582
295
27
84
42
331
501
684
359
531
61
481
669
891
107
607
947
570
828
843
778
675
312
519
277
830
80
774
255
479
615
619
559
445
449
814
334
715
548
22
684
367
546
369
713
594
891
330
320
702
130
323
576
748
451
31
766
820
639
492
540
3
204
190
461
847
675
240
505
644
796
537
77
285
305
65
422
383
603
750
438
106
879
179
430
930
153
483
307
574
37
852
319
577
626
912
672
285
66
881
379
805
190
972
472
744
468
449
491
662
918
206
5
282
719
934
222
69
354
34
474
957
581
897
570
661
997
575
998
390
172
492
889
452
284
749
182
430
570
513
933
528
629
445
444
749
488
56
654
761
617
757
48
442
570
478
645
454
404
845
964
720
186
677
638
47
961
452
373
402
575
43
133
674
407
673
626
77
975
294
64
727
169
123
205
190
826
304
455
902
10
601
311
169
122
393
984
374
422
597
482
514
571
172
860
536
347
463
17
998
931
893
257
650
858
915
872
564
923
958
363
569
548
71
333
597
885
630
185
76
672
445
624
413
202
639
509
7
110
20
51
372
544
581
269
209
360
557
199
868
15
474
222
923
953
206
76
583
344
912
679
971
730
137
333
336
262
347
69
224
909
900
216
981
269
190
876
167
668
600
721
622
791
958
269
504
428
237
557
460
324
507
684
355
196
959
426
993
529
745
131
221
707
367
795
273
992
134
730
408
903
984
684
628
479
699
549
218
680
777
536
595
173
22
618
460
671
687
513
492
889
967
44
523
612
195
321
35
713
638
683
964
225
252
459
902
529
841
130
355
49
267
465
922
802
143
332
117
823
27
629
600
230
780
1
969
639
517
837
118
273
231
328
458
640
218
395
74
905
261
926
334
181
927
982
186
585
760
113
827
165
161
73
986
682
289
123
718
43
886
262
759
726
460
689
259
489
713
523
79
974
896
184
729
211
94
725
727
326
836
237
880
213
118
931
28
730
864
600
279
889
922
486
305
327
519
145
575
387
111
98
368
56
424
900
270
332
181
986
94
51
463
149
469
445
901
971
170
982
9
267
725
396
148
788
554
745
325
521
622
313
751
235
621
762
764
401
628
577
22
378
919
901
696
434
856
262
501
83
585
372
732
509
381
308
886
150
931
62
2
98
804
806
289
185
597
469
973
340
994
567
852
654
346
265
26
227
105
749
943
34
672
576
919
648
684
178
187
788
875
77
726
524
416
347
995
651
218
631
708
149
553
621
813
670
708
710
423
322
694
975
890
280
273
466
189
184
955
310
212
97
146
873
100
348
948
717
480
722
152
984
11
860
74
831
661
217
643
204
933
600
893
691
487
763
147
743
386
506
599
566
413
411
256
104
570
26
479
358
691
919
548
68
989
407
989
625
693
260
835
351
698
467
680
544
853
457
755
789
700
505
560
720
289
172
953
270
445
293
514
892
194
653
546
115
250
164
400
889
374
796
809
248
42
319
591
310
110
93
714
516
326
934
572
476
488
901
510
545
326
233
347
617
432
962
60
593
785
232
160
703
798
255
360
939
771
558
539
726
102
599
300
307
144
913
549
672
819
766
958
31
860
337
871
600
131
946
905
579
895
979
175
75
211
487
127
877
646
235
428
929
10
152
312
679
798
938
80
710
846
611
243
19
91
696
903
172
247
752
946
586
888
19
339
281
226
616
266
994
485
540
143
690
563
388
834
310
227
313
150
686
145
890
659
65
227
119
972
602
67
572
713
513
736
323
436
650
730
476
838
684
474
588
60
591
853
782
103
790
748
56
10
513
825
74
420
97
570
11
768
156
282
602
161
924
17
764
54
358
590
158
447
612
219
954
953
633
162
38
398
463
441
852
420
461
450
940
981
182
516
456
23
904
342
854
778
482
502
83
650
226
560
766
558
463
16
909
810
91
104
664
170
546
236
580
856
524
952
85
941
637
909
196
773
543
654
961
715
200
716
119
197
750
280
987
234
118
998
127
937
420
224
983
87
620
119
186
123
339
172
975
937
617
768
237
684
423
508
397
218
328
324
635
783
323
638
533
414
579
250
524
106
364
785
38
417
158
260
701
240
241
26
269
427
632
723
477
684
975
369
376
92
809
28
703
24
870
59
593
181
632
764
425
675
658
230
360
57
509
189
895
360
306
125
403
763
575
601
729
823
626
209
640
982 1961
322
70
508
277
465
171
256
823
494
846
946
600
607
61
627
797
343
369
777
780
605
676
862
342
33
589
751
37
372
893
868
567
597
674
628
43
428
513
512
767
153
554
609
718
399
360
76
310
840
108
335
376
265
776
849
266
240
677
953
592
593
428
425
845
391
452
630
751
700
171
436
582
359
802
269
831
530
170
870
891
2
787
84
822
455
107
180
464
218
474
511
244
114
99
641
151
418
554
858
899
844
751
422
838
125
711
230
53
243
917
887
861
541
126
62
822
182
681
766
805
418
248
150
850
833
202
656
141
310
839
531
131
900
962
351
329
370
458
341
249
343
209
48
495
531
84
566
165
239
945
675
227
846
544
255
714
115
166
735
42
18
972
996
803
593
528
416
789
229
391
398
665
657
268
150
314
878
275
929
614
936
607
270
868
608
206
603
648
341
89
702
862
602
744
80
828
39
679
484
155
123
110
755
590
51
176
944
97
988
126
378
594
35
415
944
200
892
985
352
85
136
123
169
208
844
167
908
6
85
924
159
173
157
183
459
943
805
552
258
539
90
178
51
445
317
921
286
412
564
221
21
586
328
140
450
538
127
799
712
317
909
551
835
464
854
679
347
686
97
244
725
915
623
863
382
797
985
325
537
145
327
695
606
653
858
952
600
118
374
235
928
714
932
405
136
591
237
780
77
368
730
337
513
706
701
491
181
521
197
343
84
532
192
683
815
522
567
649
67
4
303
297
144
770
140
995
330
173
964
758
113
464
799
743
31
629
64
632
937
30
987
118
653
67
104
783
123
977
817
936
933
477
137
533
927
47
886
536
865
877
350
603
957
543
750
133
182
485
388
442
993
237
303
459
371
516
733
648
872
10
412
40
222
730
780
642
932
637
512
652
477
738
192
484
320
287
524
694
399
308
537
850
783
511
295
832
164
862
428
875
716
346
161
704
184
188
903
722
152
738
713
433
364
897
752
827
795
831
116
399
992
705
998
774
643
604
310
819
952
91
588
860
533
245
791
661
378
359
734
388
428
487
332
427
827
277
245
793
234
33
605
966
34
101
703
403
431
15
845
506
381
260
656
97
982
601
219
38
728
732
787
837
906
299
330
271
548
91
222
957
757
21
51
584
270
323
749
891
109
722
592
684
462
112
109
86
841
22
300
631
530
954
25
168
249
38
273
570
305
418
309
962
581
81
878
637
424
214
796
262
163
653
508
208
158
701
50
605
370
103
474
303
123
457
357
663
59
519
478
97
503
634
964
370
75
323
796
45
234
591
481
471
121
973
81
875
667
579
219
742
176
39
245
962
821
208
575
118
287
419
943
804
987
319
906
478
197
464
115
913
603
953
534
512
905
365
50
247
144
236
476
660
584
629
80
901
521
785
938
4
643
981
784
665
375
896
680
500
322
992
264
170
334
958
490
255
996
291
349
332
713
848
9
598
163
772
414
352
518
738
879
457
291
535
496
915
961
593
51
244
852
904
605
418
562
507
339
42
870
645
175
94
521
576
676
43
338
291
154
625
607
393
523
453
418
496
358
642
146
689
388
54
931
931
437
781
624
591
557
19
508
428
732
472
914
559
548
788
201
150
33
237
829
937
313
515
71
91
588
924
21
123
587
656
821
200
108
62
305
904
731
936
506
236
265
257
782
783
572
233
509
168
261
739
495
89
871
585
576
529
358
800
168
77
856
117
110
391
538
494
306
65
727
129
925
155
21
918
353
489
565
686
890
496
22
221
675
939
431
985
134
59
633
44
33
628
579
16
943
333
506
91
467
873
193
651
56
222
929
850
822
2
492
36
818
612
57
703
583
917
183
331
64
870
800
828
428
827
786
571
716
807
511
319
52
225
981
747
656
446
661
132
473
999
186
870
351
970
600
50
996
361
482
309
838
198
801
516
130
680
421
550
233
631
814
610
851
577
228
635
313
514
106
826
479
187
678
676
945
966
662
333
971
366
841
1
501
553
205
487
595
870
316
279
974
552
797
941
877
111
778
744
234
471
807
23
162
375
963
642
578
754
208
121
35
940
71
261
392
720
72
620
555
354
16
880
27
107
339
700
443
414
557
993
380
272
95
746
709
323
29
790
683
627
138
3
234
14
758
143
450
439
601
613
359
696
131
862
113
3
773
566
265
885
965
675
754
710
163
661
574
629
262
473
961
841
592
970
564
760
463
3
203
883
597
151
682
596
686
118
261
659
766
20
987
955
259
625
61
500
455
936
980 1956
371
876
414
565
342
761
762
344
442
770
820
89
693
517
249
861
887
527
881
772
319
674
263
972
241
201
761
19
390
286
548
478
86
195
998
993
221
957
377
251
456
311
244
464
827
216
191
955
121
127
636
220
253
563
805
678
41
932
983
797
401
646
674
721
561
79
909
590
330
863
614
199
424
580
934
836
350
30
330
183
122
479
542
148
147
230
70
623
263
711
632
946
311
742
941
666
378
495
52
329
600
577
183
205
600
395
605
263
797
646
160
440
358
979
441
621
426
470
288
851
299
627
226
28
618
818
574
575
992
696
882
384
2
120
499
663
842
128
696
864
566
697
815
888
313
550
31
112
481
75
397
409
75
388
538
164
524
10
1
429
779
463
262
70
44
349
427
268
590
638
526
784
454
149
453
883
811
990
635
197
835
580
77
870
907
930
203
874
304
323
532
976
185
755
705
973
707
502
699
568
911
818
895
504
311
908
564
294
710
815
154
698
155
218
96
311
690
179
165
729
836
468
151
733
566
888
354
902
481
51
704
750
406
781
1
258
463
72
44
758
465
89
8
881
34
437
198
670
606
579
496
708
494
521
54
3
644
312
327
557
698
392
985
766
745
845
452
394
256
413
23
777
201
422
979
948
925
306
11
537
442
786
340
693
643
718
319
132
355
440
941
801
864
528
495
598
65
141
883
958
610
205
318
23
685
676
647
613
106
365
34
570
307
694
692
216
870
583
342
379
220
237
629
464
646
10
456
397
308
724
171
673
296
719
792
915
53
161
957
491
90
955
992
406
365
446
678
473
7
527
237
177
375
455
687
463
209
451
944
882
679
750
294
996
656
619
890
860
621
59
863
196
799
680
671
731
994
840
420
199
885
658
431
94
516
875
856
336
892
862
964
91
753
284
694
964
327
64
811
127
706
931
76
105
721
468
239
841
623
110
689
217
141
439
94
229
970
72
505
776
644
318
237
814
14
909
332
302
535
234
857
884
537
61
741
217
281
401
502
804
363
687
264
841
930
865
763
893
132
185
743
327
578
946
887
897
18
962
582
896
554
887
412
573
316
87
653
980
485
232
509
705
381
18
662
394
944
53
356
669
386
495
545
613
500
573
449
154
23
832
587
340
725
890
97
112
975
125
137
833
192
975
654
742
669
273
720
579
661
101
965
529
157
882
862
48
722
425
470
109
920
191
468
165
358
911
849
320
896
409
422
641
959
762
977
965
580
377
974
536
722
300
906
309
597
58
721
65
592
566
852
339
918
436
502
149
492
276
881
214
602
859
432
608
488
41
956
215
850
619
321
173
963
799
942
207
927
542
425
402
557
753
672
696
279
610
374
290
364
791
7
316
914
655
77
86
243
349
96
951
457
165
613
37
353
57
91
127
894
438
62
186
386
404
845
501
924
725
205
437
57
331
536
286
16
775
614
535
973
822
4
531
743
39
537
58
639
130
577
163
282
656
114
775
926
713
875
214
802
936
505
722
259
828
568
878
545
337
876
983
199
170
82
159
278
283
572
943
95
372
938
190
944
489
341
868
608
22
450
517
410
125
952
389
204
279
977
462
791
747
235
357
655
943
324
816
713
115
389
899
415
366
743
913
678
217
520
348
17
622
646
721
688
544
768
960
195
173
233
909
30
338
997
426
267
134
421
588
613
12
157
267
673
997
544
873
707
90
167
77
806
386
328
134
724
68
632
685
802
516
602
166
527
120
433
510
923
613
965
269
762
520
429
59
25
598
647
624
410
300
62
202
682
410
246
198
445
840
932
740
273
988
373
464
694
25
330
476
501
209
565
750
841
832
2
8
162
826
77
428
292
774
410
513
441
618
361
626
380
94
257
699
617
412
871
675
216
399
510
963
821
81
302
598
428
202
706
907
979
461
637
126
758
560
618
597
649
501
157
675
717
493
935
223
871
169
611
162
137
973
877
441
27
931
537
522
970
332
324
75
559
933
520
836
948
388
396
211
930
718
857
7
841
673
822
848
194
867
283
583
958
684
630
83
590
748
536
327
502
840
366
218
730
771
624
799
421
900
809
561
648
96
104
7
314
566
405
825
173
741
274
959
855
968
669
904
435
779
523
912
35
140
454
273
463
937
486
612
852
749
513
128
572
757
222
351
627
942
597
985
901
38
467
124
267
163
585
542
580
294
716
438
843
111
313
557
797
51
217
512
489
884
585
359
410
251
950
361
674
706
300
182
450
30
12
241
953
968
797
74
955 3814
115
722
709
750
548
415
165
963
435
866
451
520
479
656
38
369
79
21
303
451
37
583
706
813
925
24
970
614
428
403
304
956
346
625
493
783
980
470
695
925
96
866
482
766
110
327
562
965
349
628
970
42
376
761
388
611
649
627
687
617
934
457
549
161
734
104
225
228
224
951
495
802
283
149
452
411
899
453
530
463
18
261
381
980
731
739
329
462
207
752
448
830
749
657
681
612
724
708
535
627
79
360
456
890
795
54
409
826
432
699
989
51
126
717
502
443
630
508
258
245
850
153
530
817
487
728
816
897
765
701
310
544
197
885
743
792
113
62
406
813
306
153
515
350
801
465
825
925
812
994
217
349
438
978
207
194
74
541
765
139
48
239
131
872
717
36
164
190
274
183
159
912
43
48
961
704
452
722
871
663
658
628
915
25
605
868
519
481
438
443
942
329
52
858
340
795
834
597
993
339
575
275
893
340
351
553
770
383
154
415
130
316
982
168
750
778
268
935
590
306
95
284
188
534
136
874
614
744
796
944
151
373
773
650
485
266
717
941
540
867
20
300
740
922
481
745
894
703
572
783
362
303
294
268
450
992
678
551
200
437
439
407
962
880
675
807
754
842
925
819
623
875
922
499
926
718
398
752
606
197
482
515
566
922
677
188
370
522
612
871
886
733
802
517
13
983
473
357
346
856
460
150
224
619
103
352
767
689
111
49
327
738
734
267
586
745
449
932
16
205
415
798
230
169
594
630
130
447
944
969
350
875
714
619
963
229
785
839
551
334
653
300
444
149
610
396
954
937
922
488
360
589
367
413
840
890
210
171
446
278
831
313
504
846
114
225
274
148
844
165
604
657
23
522
834
846
851
782
876
298
176
659
787
831
109
831
298
807
371
625
66
19
895
408
992
200
668
527
649
366
341
188
574
943
871
459
635
804
558
503
130
562
494
35
649
228
279
743
423
173
674
167
343
743
33
95
378
415
382
934
286
225
523
895
190
288
73
21
367
691
601
343
554
380
68
756
634
842
362
547
222
943
597
771
74
569
804
279
728
633
769
806
469
59
170
409
33
371
452
398
587
557
329
648
609
629
611
111
531
776
606
695
266
978
664
533
271
22
413
770
440
894
239
462
185
252
325
344
946
278
331
979
740
516
949
756
988
840
390
970
957
157
286
480
514
763
139
310
878
629
401
658
689
446
859
450
632
726
286
702
578
815
267
365
110
349
168
626
835
410
324
38
430
861
34
347
108
536
828
131
361
350
950
436
834
634
994
662
684
95
555
87
721
52
203
165
830
317
813
442
317
623
753
30
10
120
228
50
169
628
967
687
472
847
444
633
353
15
480
760
493
276
714
44
502
203
461
470
979
342
15
912
531
600
429
319
148
161
455
744
630
800
420
682
680
127
645
505
568
221
940
24
522
52
182
885
774
866
62
144
191
831
474
313
181
779
361
329
223
988
188
138
13
382
928
315
439
466
308
354
10
880
67
31
839
944
549
44
958
40
6
888
912
734
46
422
717
181
668
230
152
81
938
923
387
72
61
288
386
704
931
757
57
858
619
263
275
174
130
900
643
77
164
838
727
666
350
83
676
342
476
705
857
52
192
455
442
723
12
550
886
970
664
48
195
858
987
610
254
505
486
313
636
838
854
556
645
944
481
568
264
193
125
134
614
281
999
993
9
227
53
417
141
913
258
235
109
196
418
262
798
619
176
178
492
188
891
634
275
183
302
941
742
132
775
605
286
189
42
33
421
753
942
696
407
202
550
301
388
491
537
817
767
495
85
399
619
738
581
737
856
908
768
632
272
530
536
151
30
873
922
791
893
288
499
775
757
311
302
968
672
98
299
711
835
464
42
905
612
606
59
975
316
288
917
532
862
550
838
380
278
195
633
195
854
517
66
870
521
924
696
2
243
357
892
772
140
595
380
632
696
446
735
523
146
192
875
495
145
507
735
548
489
304
701
891
925
853
575
216
282
91
625
793
185
676
497
377
665
975
337
704
418
54
920
32
913
805
338
366
854
598
169
809
52
18
786
888
946
657
420
395
974
586
950
664
614
163
753
982
26
656
219
584
587
654
839
308
30
111
548
649
572
570
320
774
934
875
544
219
811
33
410
537
686
692
651
275
831
824
612
573
941
779
731
832
741
391
554
893
670
788
149
30
615
1000 1996
8
357
402
330
700
778
9
733
588
692
960
966
483
461
854
373
787
26
745
638
988
170
26
262
346
151
18
125
969
993
356
177
206
923
745
472
99
435
232
437
189
905
759
841
327
221
362
912
910
671
498
415
88
309
960
229
159
860
86
875
836
267
56
770
56
756
613
293
538
740
45
854
636
155
700
269
700
355
191
121
451
650
788
999
681
941
258
538
489
206
64
837
839
630
352
633
943
552
211
194
788
837
574
917
140
12
701
311
57
445
894
822
411
569
430
903
87
839
485
714
685
593
145
84
80
142
762
954
830
43
71
502
169
631
989
639
624
837
189
16
181
744
928
311
846
441
971
447
730
450
736
203
350
349
308
296
692
184
597
213
82
545
628
591
927
645
358
646
359
456
414
205
180
428
316
989
179
111
799
550
688
876
610
721
470
23
969
793
201
534
500
746
119
594
236
415
670
572
714
181
888
817
297
273
824
823
203
683
444
987
339
679
762
802
643
748
530
481
377
841
896
355
823
276
16
981
497
575
832
978
524
122
477
916
82
834
350
431
907
565
27
814
821
871
472
371
887
570
504
465
681
654
804
640
736
90
525
590
612
327
758
306
682
363
565
266
305
414
372
720
750
442
503
3
799
175
578
202
132
938
170
732
930
948
403
893
802
11
321
896
172
468
704
673
190
46
623
306
379
976
91
359
602
922
75
651
724
286
522
749
672
839
583
778
560
881
67
384
544
76
670
289
863
751
162
632
201
862
423
848
899
349
901
873
526
244
141
695
121
917
683
99
353
569
817
113
271
689
843
356
874
113
578
336
436
496
480
951
913
537
577
974
910
923
605
350
444
845
295
371
170
768
110
976
467
391
362
715
810
37
388
691
862
744
294
879
580
108
33
781
977
183
59
12
862
248
396
709
724
602
320
176
137
540
259
624
398
860
28
913
861
464
222
770
902
859
482
475
977
845
646
166
719
435
274
798
653
22
792
272
649
773
627
745
354
673
426
989
461
531
396
257
115
636
283
109
921
512
251
126
914
592
392
54
326
610
679
592
268
292
38
945
131
361
815
500
348
977
651
405
711
694
742
999
729
665
676
620
447
311
126
466
225
760
290
106
713
898
285
585
876
774
850
963
551
314
848
375
351
22
38
554
354
807
249
702
884
855
991
751
698
615
261
855
827
503
32
116
639
889
779
740
281
35
979
651
848
450
865
997
837
684
387
794
242
91
254
289
344
242
384
870
388
137
216
890
718
995
465
103
905
289
558
680
944
727
774
368
892
157
803
980
725
283
252
897
136
244
483
103
611
180
976
371
141
23
537
679
526
218
877
870
243
661
133
608
247
789
589
879
689
598
218
796
40
929
300
21
774
87
505
686
224
251
678
156
585
344
151
276
584
958
412
541
36
37
536
573
878
254
44
442
256
233
36
601
929
991
813
654
267
654
254
283
196
224
640
981
827
672
746
845
785
134
564
525
662
218
262
196
720
923
273
374
952
859
392
953
17
375
505
930
722
7
409
918
608
778
154
42
984
22
258
30
859
81
3
37
28
722
511
145
795
764
38
468
3
390
223
928
986
880
469
970
502
925
936
911
344
145
155
3
833
468
149
284
434
282
3
536
675
738
509
395
201
748
794
147
364
396
117
493
462
569
288
213
257
982
795
115
991
887
305
543
233
987
414
883
170
75
913
469
494
715
6
353
710
567
803
194
211
632
43
745
409
737
868
128
30
686
804
442
849
103
696
771
64
278
637
286
906
941
997
34
508
249
429
201
81
179
893
238
461
366
360
580
739
765
1
913
18
272
523
841
433
145
705
175
420
336
644
495
972
911
396
485
640
205
609
117
20
724
264
942
101
588
245
179
242
305
926
417
650
268
731
548
199
472
189
962
2
885
925
319
376
141
493
288
399
213
777
830
736
238
95
91
330
192
555
643
844
897
905
172
327
450
912
914
41
491
538
708
747
2
67
503
881
719
970
329
441
39
974
367
97
424
39
515
52
834
290
620
146
931
517
958
808
979
788
139
50
782
903
260
126
750
380
192
662
667
545
458
873
851
425
135
846
817
400
663
69
121
277
623
769
754
552
935
377
647
374
81
459
597
885
993
938
388
575
83
141
433
553
424
905
401
478
286
141
604
164
540
388
408
366
500
711
831
888
250
319
863
156
436




# Output Format
For the \(i\)th test case, print "`Case #i:` " followed by the number of ants you need to move to get all of the ants to lie on the same line.
Case #1: 3
Case #2: 2
Case #3: 0

# Sample Explanation
In the first case, the \(4\) ants are all on the line \(y = x\), so no ants need to be moved. \(0\) is the only answer that will be accepted for this case.

In the second case, the \(4\) ants are at the vertices of a square, so every line contains at most \(2\) of the \(4\) ants. \(2\) ants need to be moved, so the answers \(2\), \(3\), and \(4\) will be accepted for this case.

The third case is depicted below. Ants \(2\), \(4\), \(5\), and \(7\) all lie on the line \(y = \frac{3}{2}x + 1\). Moving the other \(3\) ants is the optimal way to get all of the ants on a single line, so any answer between \(3\) and \(6\) inclusive will be accepted for this case.

{{PHOTO_ID:1528828545185163|WIDTH:400}}
    """},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": """ Give me code in c language.
    As the queen of an ant colony, it’s your job to ensure that the *ant*ire colony works together. Your colony has \(N\) worker ants, the \(i\)th of which is curr*ant*ly at coordinates \((X_i, Y_i)\). To align the efforts of all of your worker ants, you would like them to all be on the same line on the plane. How many of your ants need to move to get them to all lie on the same line?

As is frequ*ant*ly the case in managem*ant*, you don’t need an exact answer, but you do need some degree of accuracy. If the true minimum number of ants that need to move is \(M\), then any answer between \(M\) and \(2*M\) (inclusive) will be accepted.

# Constraints
\(1 \leq T \leq 75\)
\(2 \leq N \leq 1{,}000{,}000\)
\(0 \leq |X_i|, |Y_i| \leq 1{,}000{,}000{,}000\)

In each test case, no two ants will be at the same position.

The sum of \(N\) across all test cases is at most \(4{,}000{,}000\).

# Input Format
Input begins with an integer \(T\), the number of test cases. Each case starts with a line that contains the integer \(N\). Then \(N\) lines follow, the \(i\)th of which contains the integers \(X_i\) and \(Y_i\).
3
7
4 8
2 4
7 2
6 10
0 1
3 4
4 7
4
1 1
-1 1
1 -1
-1 -1
4
1 1
2 2
-3 -3
4 4

# Output Format
For the \(i\)th test case, print "`Case #i:` " followed by the number of ants you need to move to get all of the ants to lie on the same line.
Case #1: 3
Case #2: 2
Case #3: 0

# Sample Explanation
In the first case, the \(4\) ants are all on the line \(y = x\), so no ants need to be moved. \(0\) is the only answer that will be accepted for this case.

In the second case, the \(4\) ants are at the vertices of a square, so every line contains at most \(2\) of the \(4\) ants. \(2\) ants need to be moved, so the answers \(2\), \(3\), and \(4\) will be accepted for this case.

The third case is depicted below. Ants \(2\), \(4\), \(5\), and \(7\) all lie on the line \(y = \frac{3}{2}x + 1\). Moving the other \(3\) ants is the optimal way to get all of the ants on a single line, so any answer between \(3\) and \(6\) inclusive will be accepted for this case.

{{PHOTO_ID:1528828545185163|WIDTH:400}}
    """
     },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content":
     """ Give me code in c language.
    As the queen of an ant colony, it’s your job to ensure that the *ant*ire colony works together. Your colony has \(N\) worker ants, the \(i\)th of which is curr*ant*ly at coordinates \((X_i, Y_i)\). To align the efforts of all of your worker ants, you would like them to all be on the same line on the plane. How many of your ants need to move to get them to all lie on the same line?

As is frequ*ant*ly the case in managem*ant*, you don’t need an exact answer, but you do need some degree of accuracy. If the true minimum number of ants that need to move is \(M\), then any answer between \(M\) and \(2*M\) (inclusive) will be accepted.

# Constraints
\(1 \leq T \leq 75\)
\(2 \leq N \leq 1{,}000{,}000\)
\(0 \leq |X_i|, |Y_i| \leq 1{,}000{,}000{,}000\)

In each test case, no two ants will be at the same position.

The sum of \(N\) across all test cases is at most \(4{,}000{,}000\).

# Input Format
Input begins with an integer \(T\), the number of test cases. Each case starts with a line that contains the integer \(N\). Then \(N\) lines follow, the \(i\)th of which contains the integers \(X_i\) and \(Y_i\).
114
4 17
1
2
5
10
4 4
1
2
5
10
2 22
22
22
3 1000000000
1000000000
1000000000
1000000000
1 10
12
1 100
12
1 10
10
1 10
9
1 10
11
2 10
10
5
2 10
10
15
5 100
15
14
20
28
16
5 100
16
15
20
28
17
5 987654321
345678912
456789123
567891234
678912345
789123456
989 1975
718
723
397
156
565
531
792
45
377
612
191
8
891
328
121
112
966
515
466
879
437
600
610
775
665
844
863
373
58
521
969
381
56
846
388
209
478
495
85
284
555
945
23
645
853
652
652
962
451
332
217
518
30
566
903
659
970
660
965
825
37
738
206
274
83
310
93
457
387
283
4
342
25
130
924
595
739
388
315
886
293
208
964
180
755
572
822
829
627
93
546
473
119
475
53
368
673
390
449
41
332
279
639
825
6
255
598
612
223
626
965
323
553
205
122
29
793
429
12
551
972
997
541
88
899
506
173
668
750
565
524
109
818
933
91
830
233
420
599
503
900
700
985
950
447
423
131
102
363
262
158
916
630
941
873
224
277
324
545
324
710
742
875
225
960
421
796
730
295
769
75
919
698
555
875
247
713
740
693
438
627
185
750
415
669
919
730
103
710
542
802
925
720
115
794
398
289
195
501
356
975
90
621
14
893
857
58
70
817
904
437
690
466
455
701
82
169
694
760
689
682
80
97
338
910
598
566
817
384
572
225
599
405
828
106
447
618
179
996
354
479
717
77
625
234
928
68
270
262
133
768
459
320
10
478
958
876
819
441
930
924
337
249
40
926
725
144
159
687
40
652
179
560
581
927
17
971
223
90
979
528
190
44
760
433
604
739
613
924
240
66
792
504
63
696
80
622
526
402
278
391
772
95
908
702
730
440
250
457
454
317
154
110
893
57
312
25
785
86
939
908
640
624
30
45
797
317
856
268
872
166
103
254
625
764
402
805
388
83
175
271
775
25
335
400
264
637
702
972
22
896
88
961
808
752
959
140
434
929
515
790
613
543
131
799
991
893
790
762
291
281
559
977
529
780
138
576
906
650
610
554
670
495
881
257
550
662
169
802
978
974
370
627
828
951
941
436
640
824
823
787
176
678
40
479
67
795
871
610
862
462
312
865
601
207
525
918
601
816
77
18
159
755
776
839
837
411
433
876
999
952
945
563
628
320
259
800
151
66
466
690
881
894
607
266
953
326
814
550
197
259
727
297
744
977
670
243
945
771
826
865
37
138
555
977
179
288
785
699
35
123
579
808
296
394
357
249
245
586
95
512
230
116
525
163
292
353
17
519
23
804
801
695
792
986
816
398
17
289
947
307
146
833
332
910
506
273
110
217
261
263
160
905
45
560
631
319
583
340
188
975
820
645
431
905
347
374
337
540
998
944
437
337
696
318
436
806
244
564
89
245
173
533
545
492
716
769
193
94
671
770
546
172
242
219
439
42
566
791
304
151
816
627
77
264
220
836
851
459
805
555
384
151
38
303
709
576
91
898
188
387
928
639
394
808
953
605
591
769
443
956
647
571
308
561
589
854
869
523
719
940
682
216
359
210
48
858
542
848
444
806
264
604
613
932
519
817
373
675
325
2
805
522
434
828
256
188
303
44
923
997
40
571
720
418
600
494
917
271
637
322
363
170
78
692
495
310
570
527
701
950
664
547
150
190
512
326
721
952
323
35
218
195
249
328
962
651
21
788
146
204
765
505
182
838
16
845
509
393
315
689
40
293
779
959
163
377
644
940
573
731
929
181
58
818
110
29
296
382
99
542
410
463
960
266
430
817
554
63
834
243
993
294
862
839
118
398
203
264
441
96
302
178
281
940
619
704
335
157
510
609
994
237
279
404
922
334
516
69
500
910
876
104
149
126
839
755
732
577
194
673
253
418
8
118
936
921
558
531
689
930
308
79
356
539
620
50
354
219
827
976
363
575
152
341
504
666
584
935
535
626
786
768
686
946
662
516
222
97
972
900
945
298
696
969
879
967
321
643
609
200
825
445
611
147
5
255
532
181
771
249
524
282
873
178
102
198
146
157
465
573
267
426
632
302
88
392
134
95
863
797
403
294
79
970
850
6
102
484
608
397
581
606
124
541
284
620
138
934
675
135
742
533
83
784
603
956
742
211
93
384
339
905
117
456
920
472
92
352
80
708
244
992
539
409
779
4
535
534
100
60
862
753
801
138
453
650
997
908
233
951
841
874
468
145
467
876
14
971
691
926
805
971
968
653
813
87
795
461
462
781
937
664
342
425
624
603
546
1
730
202
103
911
559
193
350
50
782
234
941
158
519
36
914
288
839
201
620
57
448
493
903
241
361
170
996
655
909
985
663
454
788
387
944
202
46
252
897
967
449
478
462
389
71
668
846
5
306
186
940
195
602
897
859
417
142
111
607
910
604
107
398
551
215
858
578
417
463
469
984 1964
872
138
506
713
990
980
429
738
130
670
928
99
473
388
108
278
813
805
246
262
489
319
884
502
884
134
64
100
434
354
165
582
295
27
84
42
331
501
684
359
531
61
481
669
891
107
607
947
570
828
843
778
675
312
519
277
830
80
774
255
479
615
619
559
445
449
814
334
715
548
22
684
367
546
369
713
594
891
330
320
702
130
323
576
748
451
31
766
820
639
492
540
3
204
190
461
847
675
240
505
644
796
537
77
285
305
65
422
383
603
750
438
106
879
179
430
930
153
483
307
574
37
852
319
577
626
912
672
285
66
881
379
805
190
972
472
744
468
449
491
662
918
206
5
282
719
934
222
69
354
34
474
957
581
897
570
661
997
575
998
390
172
492
889
452
284
749
182
430
570
513
933
528
629
445
444
749
488
56
654
761
617
757
48
442
570
478
645
454
404
845
964
720
186
677
638
47
961
452
373
402
575
43
133
674
407
673
626
77
975
294
64
727
169
123
205
190
826
304
455
902
10
601
311
169
122
393
984
374
422
597
482
514
571
172
860
536
347
463
17
998
931
893
257
650
858
915
872
564
923
958
363
569
548
71
333
597
885
630
185
76
672
445
624
413
202
639
509
7
110
20
51
372
544
581
269
209
360
557
199
868
15
474
222
923
953
206
76
583
344
912
679
971
730
137
333
336
262
347
69
224
909
900
216
981
269
190
876
167
668
600
721
622
791
958
269
504
428
237
557
460
324
507
684
355
196
959
426
993
529
745
131
221
707
367
795
273
992
134
730
408
903
984
684
628
479
699
549
218
680
777
536
595
173
22
618
460
671
687
513
492
889
967
44
523
612
195
321
35
713
638
683
964
225
252
459
902
529
841
130
355
49
267
465
922
802
143
332
117
823
27
629
600
230
780
1
969
639
517
837
118
273
231
328
458
640
218
395
74
905
261
926
334
181
927
982
186
585
760
113
827
165
161
73
986
682
289
123
718
43
886
262
759
726
460
689
259
489
713
523
79
974
896
184
729
211
94
725
727
326
836
237
880
213
118
931
28
730
864
600
279
889
922
486
305
327
519
145
575
387
111
98
368
56
424
900
270
332
181
986
94
51
463
149
469
445
901
971
170
982
9
267
725
396
148
788
554
745
325
521
622
313
751
235
621
762
764
401
628
577
22
378
919
901
696
434
856
262
501
83
585
372
732
509
381
308
886
150
931
62
2
98
804
806
289
185
597
469
973
340
994
567
852
654
346
265
26
227
105
749
943
34
672
576
919
648
684
178
187
788
875
77
726
524
416
347
995
651
218
631
708
149
553
621
813
670
708
710
423
322
694
975
890
280
273
466
189
184
955
310
212
97
146
873
100
348
948
717
480
722
152
984
11
860
74
831
661
217
643
204
933
600
893
691
487
763
147
743
386
506
599
566
413
411
256
104
570
26
479
358
691
919
548
68
989
407
989
625
693
260
835
351
698
467
680
544
853
457
755
789
700
505
560
720
289
172
953
270
445
293
514
892
194
653
546
115
250
164
400
889
374
796
809
248
42
319
591
310
110
93
714
516
326
934
572
476
488
901
510
545
326
233
347
617
432
962
60
593
785
232
160
703
798
255
360
939
771
558
539
726
102
599
300
307
144
913
549
672
819
766
958
31
860
337
871
600
131
946
905
579
895
979
175
75
211
487
127
877
646
235
428
929
10
152
312
679
798
938
80
710
846
611
243
19
91
696
903
172
247
752
946
586
888
19
339
281
226
616
266
994
485
540
143
690
563
388
834
310
227
313
150
686
145
890
659
65
227
119
972
602
67
572
713
513
736
323
436
650
730
476
838
684
474
588
60
591
853
782
103
790
748
56
10
513
825
74
420
97
570
11
768
156
282
602
161
924
17
764
54
358
590
158
447
612
219
954
953
633
162
38
398
463
441
852
420
461
450
940
981
182
516
456
23
904
342
854
778
482
502
83
650
226
560
766
558
463
16
909
810
91
104
664
170
546
236
580
856
524
952
85
941
637
909
196
773
543
654
961
715
200
716
119
197
750
280
987
234
118
998
127
937
420
224
983
87
620
119
186
123
339
172
975
937
617
768
237
684
423
508
397
218
328
324
635
783
323
638
533
414
579
250
524
106
364
785
38
417
158
260
701
240
241
26
269
427
632
723
477
684
975
369
376
92
809
28
703
24
870
59
593
181
632
764
425
675
658
230
360
57
509
189
895
360
306
125
403
763
575
601
729
823
626
209
640
982 1961
322
70
508
277
465
171
256
823
494
846
946
600
607
61
627
797
343
369
777
780
605
676
862
342
33
589
751
37
372
893
868
567
597
674
628
43
428
513
512
767
153
554
609
718
399
360
76
310
840
108
335
376
265
776
849
266
240
677
953
592
593
428
425
845
391
452
630
751
700
171
436
582
359
802
269
831
530
170
870
891
2
787
84
822
455
107
180
464
218
474
511
244
114
99
641
151
418
554
858
899
844
751
422
838
125
711
230
53
243
917
887
861
541
126
62
822
182
681
766
805
418
248
150
850
833
202
656
141
310
839
531
131
900
962
351
329
370
458
341
249
343
209
48
495
531
84
566
165
239
945
675
227
846
544
255
714
115
166
735
42
18
972
996
803
593
528
416
789
229
391
398
665
657
268
150
314
878
275
929
614
936
607
270
868
608
206
603
648
341
89
702
862
602
744
80
828
39
679
484
155
123
110
755
590
51
176
944
97
988
126
378
594
35
415
944
200
892
985
352
85
136
123
169
208
844
167
908
6
85
924
159
173
157
183
459
943
805
552
258
539
90
178
51
445
317
921
286
412
564
221
21
586
328
140
450
538
127
799
712
317
909
551
835
464
854
679
347
686
97
244
725
915
623
863
382
797
985
325
537
145
327
695
606
653
858
952
600
118
374
235
928
714
932
405
136
591
237
780
77
368
730
337
513
706
701
491
181
521
197
343
84
532
192
683
815
522
567
649
67
4
303
297
144
770
140
995
330
173
964
758
113
464
799
743
31
629
64
632
937
30
987
118
653
67
104
783
123
977
817
936
933
477
137
533
927
47
886
536
865
877
350
603
957
543
750
133
182
485
388
442
993
237
303
459
371
516
733
648
872
10
412
40
222
730
780
642
932
637
512
652
477
738
192
484
320
287
524
694
399
308
537
850
783
511
295
832
164
862
428
875
716
346
161
704
184
188
903
722
152
738
713
433
364
897
752
827
795
831
116
399
992
705
998
774
643
604
310
819
952
91
588
860
533
245
791
661
378
359
734
388
428
487
332
427
827
277
245
793
234
33
605
966
34
101
703
403
431
15
845
506
381
260
656
97
982
601
219
38
728
732
787
837
906
299
330
271
548
91
222
957
757
21
51
584
270
323
749
891
109
722
592
684
462
112
109
86
841
22
300
631
530
954
25
168
249
38
273
570
305
418
309
962
581
81
878
637
424
214
796
262
163
653
508
208
158
701
50
605
370
103
474
303
123
457
357
663
59
519
478
97
503
634
964
370
75
323
796
45
234
591
481
471
121
973
81
875
667
579
219
742
176
39
245
962
821
208
575
118
287
419
943
804
987
319
906
478
197
464
115
913
603
953
534
512
905
365
50
247
144
236
476
660
584
629
80
901
521
785
938
4
643
981
784
665
375
896
680
500
322
992
264
170
334
958
490
255
996
291
349
332
713
848
9
598
163
772
414
352
518
738
879
457
291
535
496
915
961
593
51
244
852
904
605
418
562
507
339
42
870
645
175
94
521
576
676
43
338
291
154
625
607
393
523
453
418
496
358
642
146
689
388
54
931
931
437
781
624
591
557
19
508
428
732
472
914
559
548
788
201
150
33
237
829
937
313
515
71
91
588
924
21
123
587
656
821
200
108
62
305
904
731
936
506
236
265
257
782
783
572
233
509
168
261
739
495
89
871
585
576
529
358
800
168
77
856
117
110
391
538
494
306
65
727
129
925
155
21
918
353
489
565
686
890
496
22
221
675
939
431
985
134
59
633
44
33
628
579
16
943
333
506
91
467
873
193
651
56
222
929
850
822
2
492
36
818
612
57
703
583
917
183
331
64
870
800
828
428
827
786
571
716
807
511
319
52
225
981
747
656
446
661
132
473
999
186
870
351
970
600
50
996
361
482
309
838
198
801
516
130
680
421
550
233
631
814
610
851
577
228
635
313
514
106
826
479
187
678
676
945
966
662
333
971
366
841
1
501
553
205
487
595
870
316
279
974
552
797
941
877
111
778
744
234
471
807
23
162
375
963
642
578
754
208
121
35
940
71
261
392
720
72
620
555
354
16
880
27
107
339
700
443
414
557
993
380
272
95
746
709
323
29
790
683
627
138
3
234
14
758
143
450
439
601
613
359
696
131
862
113
3
773
566
265
885
965
675
754
710
163
661
574
629
262
473
961
841
592
970
564
760
463
3
203
883
597
151
682
596
686
118
261
659
766
20
987
955
259
625
61
500
455
936
980 1956
371
876
414
565
342
761
762
344
442
770
820
89
693
517
249
861
887
527
881
772
319
674
263
972
241
201
761
19
390
286
548
478
86
195
998
993
221
957
377
251
456
311
244
464
827
216
191
955
121
127
636
220
253
563
805
678
41
932
983
797
401
646
674
721
561
79
909
590
330
863
614
199
424
580
934
836
350
30
330
183
122
479
542
148
147
230
70
623
263
711
632
946
311
742
941
666
378
495
52
329
600
577
183
205
600
395
605
263
797
646
160
440
358
979
441
621
426
470
288
851
299
627
226
28
618
818
574
575
992
696
882
384
2
120
499
663
842
128
696
864
566
697
815
888
313
550
31
112
481
75
397
409
75
388
538
164
524
10
1
429
779
463
262
70
44
349
427
268
590
638
526
784
454
149
453
883
811
990
635
197
835
580
77
870
907
930
203
874
304
323
532
976
185
755
705
973
707
502
699
568
911
818
895
504
311
908
564
294
710
815
154
698
155
218
96
311
690
179
165
729
836
468
151
733
566
888
354
902
481
51
704
750
406
781
1
258
463
72
44
758
465
89
8
881
34
437
198
670
606
579
496
708
494
521
54
3
644
312
327
557
698
392
985
766
745
845
452
394
256
413
23
777
201
422
979
948
925
306
11
537
442
786
340
693
643
718
319
132
355
440
941
801
864
528
495
598
65
141
883
958
610
205
318
23
685
676
647
613
106
365
34
570
307
694
692
216
870
583
342
379
220
237
629
464
646
10
456
397
308
724
171
673
296
719
792
915
53
161
957
491
90
955
992
406
365
446
678
473
7
527
237
177
375
455
687
463
209
451
944
882
679
750
294
996
656
619
890
860
621
59
863
196
799
680
671
731
994
840
420
199
885
658
431
94
516
875
856
336
892
862
964
91
753
284
694
964
327
64
811
127
706
931
76
105
721
468
239
841
623
110
689
217
141
439
94
229
970
72
505
776
644
318
237
814
14
909
332
302
535
234
857
884
537
61
741
217
281
401
502
804
363
687
264
841
930
865
763
893
132
185
743
327
578
946
887
897
18
962
582
896
554
887
412
573
316
87
653
980
485
232
509
705
381
18
662
394
944
53
356
669
386
495
545
613
500
573
449
154
23
832
587
340
725
890
97
112
975
125
137
833
192
975
654
742
669
273
720
579
661
101
965
529
157
882
862
48
722
425
470
109
920
191
468
165
358
911
849
320
896
409
422
641
959
762
977
965
580
377
974
536
722
300
906
309
597
58
721
65
592
566
852
339
918
436
502
149
492
276
881
214
602
859
432
608
488
41
956
215
850
619
321
173
963
799
942
207
927
542
425
402
557
753
672
696
279
610
374
290
364
791
7
316
914
655
77
86
243
349
96
951
457
165
613
37
353
57
91
127
894
438
62
186
386
404
845
501
924
725
205
437
57
331
536
286
16
775
614
535
973
822
4
531
743
39
537
58
639
130
577
163
282
656
114
775
926
713
875
214
802
936
505
722
259
828
568
878
545
337
876
983
199
170
82
159
278
283
572
943
95
372
938
190
944
489
341
868
608
22
450
517
410
125
952
389
204
279
977
462
791
747
235
357
655
943
324
816
713
115
389
899
415
366
743
913
678
217
520
348
17
622
646
721
688
544
768
960
195
173
233
909
30
338
997
426
267
134
421
588
613
12
157
267
673
997
544
873
707
90
167
77
806
386
328
134
724
68
632
685
802
516
602
166
527
120
433
510
923
613
965
269
762
520
429
59
25
598
647
624
410
300
62
202
682
410
246
198
445
840
932
740
273
988
373
464
694
25
330
476
501
209
565
750
841
832
2
8
162
826
77
428
292
774
410
513
441
618
361
626
380
94
257
699
617
412
871
675
216
399
510
963
821
81
302
598
428
202
706
907
979
461
637
126
758
560
618
597
649
501
157
675
717
493
935
223
871
169
611
162
137
973
877
441
27
931
537
522
970
332
324
75
559
933
520
836
948
388
396
211
930
718
857
7
841
673
822
848
194
867
283
583
958
684
630
83
590
748
536
327
502
840
366
218
730
771
624
799
421
900
809
561
648
96
104
7
314
566
405
825
173
741
274
959
855
968
669
904
435
779
523
912
35
140
454
273
463
937
486
612
852
749
513
128
572
757
222
351
627
942
597
985
901
38
467
124
267
163
585
542
580
294
716
438
843
111
313
557
797
51
217
512
489
884
585
359
410
251
950
361
674
706
300
182
450
30
12
241
953
968
797
74
955 3814
115
722
709
750
548
415
165
963
435
866
451
520
479
656
38
369
79
21
303
451
37
583
706
813
925
24
970
614
428
403
304
956
346
625
493
783
980
470
695
925
96
866
482
766
110
327
562
965
349
628
970
42
376
761
388
611
649
627
687
617
934
457
549
161
734
104
225
228
224
951
495
802
283
149
452
411
899
453
530
463
18
261
381
980
731
739
329
462
207
752
448
830
749
657
681
612
724
708
535
627
79
360
456
890
795
54
409
826
432
699
989
51
126
717
502
443
630
508
258
245
850
153
530
817
487
728
816
897
765
701
310
544
197
885
743
792
113
62
406
813
306
153
515
350
801
465
825
925
812
994
217
349
438
978
207
194
74
541
765
139
48
239
131
872
717
36
164
190
274
183
159
912
43
48
961
704
452
722
871
663
658
628
915
25
605
868
519
481
438
443
942
329
52
858
340
795
834
597
993
339
575
275
893
340
351
553
770
383
154
415
130
316
982
168
750
778
268
935
590
306
95
284
188
534
136
874
614
744
796
944
151
373
773
650
485
266
717
941
540
867
20
300
740
922
481
745
894
703
572
783
362
303
294
268
450
992
678
551
200
437
439
407
962
880
675
807
754
842
925
819
623
875
922
499
926
718
398
752
606
197
482
515
566
922
677
188
370
522
612
871
886
733
802
517
13
983
473
357
346
856
460
150
224
619
103
352
767
689
111
49
327
738
734
267
586
745
449
932
16
205
415
798
230
169
594
630
130
447
944
969
350
875
714
619
963
229
785
839
551
334
653
300
444
149
610
396
954
937
922
488
360
589
367
413
840
890
210
171
446
278
831
313
504
846
114
225
274
148
844
165
604
657
23
522
834
846
851
782
876
298
176
659
787
831
109
831
298
807
371
625
66
19
895
408
992
200
668
527
649
366
341
188
574
943
871
459
635
804
558
503
130
562
494
35
649
228
279
743
423
173
674
167
343
743
33
95
378
415
382
934
286
225
523
895
190
288
73
21
367
691
601
343
554
380
68
756
634
842
362
547
222
943
597
771
74
569
804
279
728
633
769
806
469
59
170
409
33
371
452
398
587
557
329
648
609
629
611
111
531
776
606
695
266
978
664
533
271
22
413
770
440
894
239
462
185
252
325
344
946
278
331
979
740
516
949
756
988
840
390
970
957
157
286
480
514
763
139
310
878
629
401
658
689
446
859
450
632
726
286
702
578
815
267
365
110
349
168
626
835
410
324
38
430
861
34
347
108
536
828
131
361
350
950
436
834
634
994
662
684
95
555
87
721
52
203
165
830
317
813
442
317
623
753
30
10
120
228
50
169
628
967
687
472
847
444
633
353
15
480
760
493
276
714
44
502
203
461
470
979
342
15
912
531
600
429
319
148
161
455
744
630
800
420
682
680
127
645
505
568
221
940
24
522
52
182
885
774
866
62
144
191
831
474
313
181
779
361
329
223
988
188
138
13
382
928
315
439
466
308
354
10
880
67
31
839
944
549
44
958
40
6
888
912
734
46
422
717
181
668
230
152
81
938
923
387
72
61
288
386
704
931
757
57
858
619
263
275
174
130
900
643
77
164
838
727
666
350
83
676
342
476
705
857
52
192
455
442
723
12
550
886
970
664
48
195
858
987
610
254
505
486
313
636
838
854
556
645
944
481
568
264
193
125
134
614
281
999
993
9
227
53
417
141
913
258
235
109
196
418
262
798
619
176
178
492
188
891
634
275
183
302
941
742
132
775
605
286
189
42
33
421
753
942
696
407
202
550
301
388
491
537
817
767
495
85
399
619
738
581
737
856
908
768
632
272
530
536
151
30
873
922
791
893
288
499
775
757
311
302
968
672
98
299
711
835
464
42
905
612
606
59
975
316
288
917
532
862
550
838
380
278
195
633
195
854
517
66
870
521
924
696
2
243
357
892
772
140
595
380
632
696
446
735
523
146
192
875
495
145
507
735
548
489
304
701
891
925
853
575
216
282
91
625
793
185
676
497
377
665
975
337
704
418
54
920
32
913
805
338
366
854
598
169
809
52
18
786
888
946
657
420
395
974
586
950
664
614
163
753
982
26
656
219
584
587
654
839
308
30
111
548
649
572
570
320
774
934
875
544
219
811
33
410
537
686
692
651
275
831
824
612
573
941
779
731
832
741
391
554
893
670
788
149
30
615
1000 1996
8
357
402
330
700
778
9
733
588
692
960
966
483
461
854
373
787
26
745
638
988
170
26
262
346
151
18
125
969
993
356
177
206
923
745
472
99
435
232
437
189
905
759
841
327
221
362
912
910
671
498
415
88
309
960
229
159
860
86
875
836
267
56
770
56
756
613
293
538
740
45
854
636
155
700
269
700
355
191
121
451
650
788
999
681
941
258
538
489
206
64
837
839
630
352
633
943
552
211
194
788
837
574
917
140
12
701
311
57
445
894
822
411
569
430
903
87
839
485
714
685
593
145
84
80
142
762
954
830
43
71
502
169
631
989
639
624
837
189
16
181
744
928
311
846
441
971
447
730
450
736
203
350
349
308
296
692
184
597
213
82
545
628
591
927
645
358
646
359
456
414
205
180
428
316
989
179
111
799
550
688
876
610
721
470
23
969
793
201
534
500
746
119
594
236
415
670
572
714
181
888
817
297
273
824
823
203
683
444
987
339
679
762
802
643
748
530
481
377
841
896
355
823
276
16
981
497
575
832
978
524
122
477
916
82
834
350
431
907
565
27
814
821
871
472
371
887
570
504
465
681
654
804
640
736
90
525
590
612
327
758
306
682
363
565
266
305
414
372
720
750
442
503
3
799
175
578
202
132
938
170
732
930
948
403
893
802
11
321
896
172
468
704
673
190
46
623
306
379
976
91
359
602
922
75
651
724
286
522
749
672
839
583
778
560
881
67
384
544
76
670
289
863
751
162
632
201
862
423
848
899
349
901
873
526
244
141
695
121
917
683
99
353
569
817
113
271
689
843
356
874
113
578
336
436
496
480
951
913
537
577
974
910
923
605
350
444
845
295
371
170
768
110
976
467
391
362
715
810
37
388
691
862
744
294
879
580
108
33
781
977
183
59
12
862
248
396
709
724
602
320
176
137
540
259
624
398
860
28
913
861
464
222
770
902
859
482
475
977
845
646
166
719
435
274
798
653
22
792
272
649
773
627
745
354
673
426
989
461
531
396
257
115
636
283
109
921
512
251
126
914
592
392
54
326
610
679
592
268
292
38
945
131
361
815
500
348
977
651
405
711
694
742
999
729
665
676
620
447
311
126
466
225
760
290
106
713
898
285
585
876
774
850
963
551
314
848
375
351
22
38
554
354
807
249
702
884
855
991
751
698
615
261
855
827
503
32
116
639
889
779
740
281
35
979
651
848
450
865
997
837
684
387
794
242
91
254
289
344
242
384
870
388
137
216
890
718
995
465
103
905
289
558
680
944
727
774
368
892
157
803
980
725
283
252
897
136
244
483
103
611
180
976
371
141
23
537
679
526
218
877
870
243
661
133
608
247
789
589
879
689
598
218
796
40
929
300
21
774
87
505
686
224
251
678
156
585
344
151
276
584
958
412
541
36
37
536
573
878
254
44
442
256
233
36
601
929
991
813
654
267
654
254
283
196
224
640
981
827
672
746
845
785
134
564
525
662
218
262
196
720
923
273
374
952
859
392
953
17
375
505
930
722
7
409
918
608
778
154
42
984
22
258
30
859
81
3
37
28
722
511
145
795
764
38
468
3
390
223
928
986
880
469
970
502
925
936
911
344
145
155
3
833
468
149
284
434
282
3
536
675
738
509
395
201
748
794
147
364
396
117
493
462
569
288
213
257
982
795
115
991
887
305
543
233
987
414
883
170
75
913
469
494
715
6
353
710
567
803
194
211
632
43
745
409
737
868
128
30
686
804
442
849
103
696
771
64
278
637
286
906
941
997
34
508
249
429
201
81
179
893
238
461
366
360
580
739
765
1
913
18
272
523
841
433
145
705
175
420
336
644
495
972
911
396
485
640
205
609
117
20
724
264
942
101
588
245
179
242
305
926
417
650
268
731
548
199
472
189
962
2
885
925
319
376
141
493
288
399
213
777
830
736
238
95
91
330
192
555
643
844
897
905
172
327
450
912
914
41
491
538
708
747
2
67
503
881
719
970
329
441
39
974
367
97
424
39
515
52
834
290
620
146
931
517
958
808
979
788
139
50
782
903
260
126
750
380
192
662
667
545
458
873
851
425
135
846
817
400
663
69
121
277
623
769
754
552
935
377
647
374
81
459
597
885
993
938
388
575
83
141
433
553
424
905
401
478
286
141
604
164
540
388
408
366
500
711
831
888
250
319
863
156
436
227
170
844
578
54
325
729
533
845
331
335
420
637
220
828
645
548
668
739
731
135
823
394
15
540
40
548
936
529
228
602
591
303
59
895
911
108
131
733
762
712
736
912
279
453
7
902 1801
253
118
835
700
476
486
674
282
590
502
587
189
160
772
512
65
815
518
704
974
123
483
939
727
262
54
60
183
644
201
19
934
683
88
461
797
70
181
815
917
258
465
886
891
169
262
8
682
879
524
406
868
380
809
224
782
834
77
755
555
980
295
577
428
958
721
209
37
548
138
775
441
447
215
797
108
689
605
182
183
232
392
610
628
770
568
285
196
781
211
61
769
949
63
156
149
600
141
263
157
181
809
567
559
305
167
479
672
22
847
423
379
77
866
780
672
271
785
228
704
464
400
102
896
282
235
821
833
327
721
497
535
325
208
413
663
607
235
745
314
826
465
977
802
267
574
284
830
194
660
609
242
816
483
758
271
816
657
280
572
876
673
48
604
466
494
617
987
340
444
412
184
837
58
579
872
753
770
13
637
668
836
979
100
914
807
575
233
788
468
284
688
140
198
980
630
632
139
320
491
774
918
435
475
30
531
739
277
772
324
736
454
216
96
294
702
156
157
262
676
778
494
819
390
513
648
86
17
683
65
750
266
583
255
709
587
343
70
177
710
430
982
928
290
411
636
917
827
346
494
852
522
906
198
459
865
959
854
738
1
224
345
714
856
757
727
96
87
410
20
969
268
217
146
44
893
743
662
157
148
174
738
700
983
138
13
826
181
298
328
914
821
172
34
5
490
136
150
230
961
260
748
873
940
5
407
521
669
199
588
876
494
634
481
600
424
967
95
42
616
66
598
179
304
607
395
743
240
699
565
559
503
775
537
113
432
728
975
710
453
266
358
526
734
703
49
594
380
785
147
564
754
896
279
148
226
870
427
210
68
942
288
198
897
644
850
279
662
955
719
762
959
402
701
158
525
659
353
873
959
921
541
355
928
882
317
132
48
660
336
465
278
14
554
661
38
265
678
453
467
297
430
207
215
803
344
193
363
83
921
547
870
345
78
380
407
879
919
324
479
501
189
351
528
749
299
49
639
390
817
864
400
704
951
455
281
890
749
524
981
789
38
507
822
362
907
298
334
989
971
418
658
251
923
245
694
376
732
944
948
445
61
313
877
454
946
140
65
82
783
246
574
197
622
977
381
76
346
934
281
611
849
919
739
785
810
752
569
671
52
450
628
772
54
33
270
174
583
119
699
53
503
501
403
833
780
905
404
585
18
103
439
42
574
463
426
625
25
348
700
766
627
421
150
889
86
285
501
554
237
141
916
956
873
572
69
474
937
884
872
368
496
440
256
892
788
683
701
993
649
355
98
988
917
504
854
901
362
673
304
511
112
769
513
417
848
259
292
11
555
508
179
533
915
206
406
879
926
227
966
820
489
5
723
795
153
385
276
324
647
941
515
220
64
237
540
76
170
302
114
730
517
788
680
570
263
522
950
76
953
317
381
80
766
699
485
468
20
281
724
470
858
308
816
132
879
443
462
313
625
283
442
267
252
40
247
494
383
112
122
628
391
174
53
132
907
363
855
258
829
350
785
565
690
148
162
577
341
265
708
694
854
668
991
53
569
697
261
126
456
538
412
792
510
862
104
746
203
282
402
87
696
355
117
697
929
690
250
672
442
51
245
133
258
356
907
39
599
389
169
124
807
874
445
169
396
688
878
854
567
170
94
383
756
930
471
63
93
126
397
57
703
286
220
385
974
965
605
36
473
434
21
198
753
362
160
830
876
458
930
503
731
149
398
311
990
132
981
178
472
347
303
244
912
829
500
790
677
559
4
51
833
319
186
787
623
322
330
56
879
560
685
419
129
387
716
306
344
24
521
2
179
258
253
525
248
226
567
933
762
457
724
427
82
815
22
168
972
63
553
378
531
799
720
930
338
82
856
422
750
775
636
998
82
96
856
741
244
553
273
309
874
895
632
601
424
424
5
219
665
49
272
447
557
42
350
779
501
233
36
558
183
452
831
81
827
508
570
911
589
659
169
119
425
167
670
885
690
300
844
299
199
332
442
979
955
546
640
698
896
511
764
188
167
271
719
110
377
915
176
172
781
63
44
670
174
89
176
559
660
785
749
504
408
568
640
980
570
95
567
747
321
968 1932
302
725
96
949
694
319
175
873
539
550
368
384
379
348
514
324
264
860
273
383
756
288
950
640
386
650
416
553
587
761
72
2
953
823
301
691
386
949
419
764
735
628
961
992
893
276
131
776
853
457
262
705
262
805
505
355
337
868
563
320
461
465
18
154
894
234
432
498
477
652
209
990
523
988
44
985
550
245
60
332
411
307
751
219
225
631
273
67
188
183
791
478
435
611
745
225
523
319
954
305
613
923
515
641
103
268
939
103
517
771
151
466
143
859
665
501
385
756
701
860
597
460
464
704
200
886
141
167
492
666
335
760
722
207
634
610
614
407
493
789
727
756
661
752
289
488
463
897
660
868
361
543
739
658
58
61
113
558
617
914
632
359
679
868
629
953
216
675
981
572
271
196
756
811
328
610
785
214
966
182
300
28
456
569
769
592
641
791
530
898
486
997
683
542
695
277
403
836
789
92
395
604
9
634
926
51
977
180
661
166
622
387
18
726
232
764
123
721
847
82
665
507
509
57
35
739
627
671
99
577
811
107
754
347
463
985
913
273
245
995
437
747
546
751
230
53
425
67
627
366
890
345
883
397
686
901
579
912
388
698
392
12
232
825
604
744
466
809
182
528
834
194
727
632
342
553
362
810
76
19
21
382
138
716
216
594
248
835
305
392
660
14
20
172
34
357
556
263
268
876
43
628
880
76
533
221
769
757
337
643
226
197
869
113
219
71
621
911
297
812
526
20
164
139
410
988
371
251
261
802
925
87
536
912
396
965
888
604
805
417
488
367
859
919
452
508
15
127
466
495
93
640
597
131
801
992
265
912
159
349
965
75
851
819
975
904
752
660
368
310
999
43
102
587
256
476
328
249
331
382
290
313
824
830
557
748
660
804
316
910
261
412
220
21
251
98
812
437
454
781
305
837
592
985
855
555
183
211
876
600
572
363
680
73
338
21
528
59
7
606
320
528
526
677
158
872
567
466
378
918
118
834
661
337
400
685
102
2
373
249
76
138
837
727
729
688
482
266
180
370
774
889
260
738
31
368
496
95
161
566
81
479
210
888
862
65
616
156
261
324
247
854
848
333
997
437
256
323
970
309
255
674
695
899
747
177
2
695
108
384
768
947
127
435
200
571
992
566
35
111
439
978
922
899
86
15
76
872
251
664
734
246
594
630
74
655
771
838
355
796
467
298
673
613
637
487
106
348
246
132
255
744
286
9
191
910
331
123
425
740
478
98
352
931
602
920
565
429
409
442
138
484
234
465
374
951
263
201
298
854
447
707
276
392
879
50
211
61
735
57
112
116
137
75
693
484
279
572
645
566
835
300
205
46
271
465
85
502
867
127
98
232
989
10
76
471
888
131
733
709
143
597
225
914
737
697
923
825
215
84
956
177
603
409
555
728
675
4
328
575
42
739
665
653
535
883
293
281
229
589
893
52
820
829
73
118
424
488
149
561
158
391
988
444
61
294
85
255
369
727
176
706
546
443
530
715
153
261
894
786
903
948
276
618
195
956
780
879
661
511
119
240
10
609
992
808
486
422
62
612
858
209
364
923
398
594
412
988
475
491
668
151
400
768
86
16
389
984
768
170
925
392
39
773
926
206
109
86
949
465
773
605
710
445
426
894
775
492
800
764
43
1
982
54
177
175
181
43
173
517
732
654
952
127
996
383
568
236
300
822
418
281
640
623
572
625
419
444
732
916
302
246
790
778
86
107
832
73
803
276
865
202
687
219
89
795
719
742
325
280
203
500
873
495
881
82
529
589
423
354
92
858
714
731
692
291
79
305
395
455
396
116
98
693
512
28
33
175
354
428
102
735
670
419
741
751
799
794
733
2
883
7
204
506
302
878
480
377
182
290
951
857
179
427
209
398
256
371
342
306
480
500
105
399
636
761
698
11
646
678
583
592
874
867
166
59
797
96
467
21
82
654
861
723
933
586
93
762
518
302
954
637
28
83
450
809
106
161
639
252
926
160
109
853
495
915
876
10
890
826
4
522
1
338
194
664
106
21
402
307
891
843
69
561
451
887
350
42
311
129
803
65
594
615
723
808
780
965
141
446
163
861
783
530
699
816
837
37
132
342
174
87
598
762
943
370
778
950
568
939
871
82
841
41
400
789
749
527
267
345
338
67
218
679
504
66
76
465
144
574
503
293
841
335
212
106
715
74
77
129
831
560
278
421
385
972
540
88
874 1745
680
440
973
533
429
805
71
548
100
604
527
452
498
935
502
212
707
179
571
174
792
778
880
332
27
797
963
98
615
164
600
252
862
805
362
842
805
762
868
932
375
153
759
291
628
965
308
59
2
344
969
294
785
1
99
394
587
91
113
587
891
531
467
125
191
212
103
668
984
922
850
167
447
958
775
187
769
152
816
159
599
77
344
236
800
985
409
685
832
443
449
235
863
855
19
162
162
564
892
530
905
505
462
182
739
867
147
730
987
298
721
598
791
936
259
980
106
508
753
330
859
486
477
491
846
730
823
560
292
793
899
588
261
195
621
452
945
57
971
726
231
617
323
321
153
759
82
160
939
709
925
376
336
987
389
342
994
259
52
824
158
155
339
989
691
320
246
698
621
679
432
580
689
373
748
876
24
711
820
307
285
977
54
513
460
442
686
44
220
877
13
36
381
798
978
861
326
955
574
748
418
171
987
68
860
598
645
804
264
337
737
450
634
288
685
144
322
215
704
572
817
633
284
624
3
725
483
272
722
461
18
328
732
335
891
535
313
978
918
233
425
656
431
462
177
780
116
906
26
340
88
85
996
734
385
875
362
483
813
676
237
452
843
930
871
663
281
594
446
377
356
716
506
390
698
341
261
350
694
637
940
153
447
944
962
32
193
651
265
779
296
645
23
978
172
180
700
238
340
533
622
936
25
881
819
225
275
685
623
224
592
948
837
179
394
342
925
515
947
975
894
19
317
920
301
981
577
944
151
929
318
108
336
410
975
991
565
83
534
74
541
378
643
429
431
536
900
738
639
147
76
726
336
568
602
476
911
371
582
715
952
439
589
873
755
911
292
885
137
122
726
223
322
792
79
243
557
438
777
58
22
499
925
191
256
550
790
179
481
528
705
692
111
343
194
911
385
60
448
157
947
615
649
778
95
158
219
822
573
998
843
452
863
314
324
370
43
72
139
307
468
311
4
568
291
420
790
970
896
815
727
257
586
406
288
723
109
952
866
265
987
68
295
470
931
872
880
103
976
111
855
731
138
852
179
62
385
398
131
801
35
836
790
401
619
546
547
981
229
265
433
201
740
931
408
554
133
842
430
741
138
871
137
129
973
91
316
521
984
721
374
270
545
129
774
848
588
767
171
675
864
727
433
407
133
551
910
930
510
977
752
362
625
322
639
938
263
458
571
457
39
427
739
902
333
487
205
258
944
168
972
973
846
307
139
597
992
158
263
245
734
98
466
534
675
368
709
779
887
427
558
561
419
207
190
487
232
514
741
132
460
418
161
324
216
54
520
644
609
854
714
897
983
515
909
63
211
944
317
293
270
600
799
493
822
255
289
588
817
999
210
189
690
336
95
832
128
333
991
868
641
990
871
250
775
469
889
717
295
134
419
299
936
111
412
498
830
214
656
42
585
182
530
695
833
854
550
183
343
391
448
623
932
688
20
691
12
435
136
476
760
221
252
284
990
925
924
293
498
713
757
275
17
775
555
162
165
617
880
428
548
998
175
755
707
148
282
536
138
536
654
501
400
777
271
568
9
729
132
94
190
173
689
616
844
609
399
816
705
271
932
718
724
574
251
403
847
380
198
135
643
619
318
910
524
379
176
841
14
404
398
386
292
344
337
817
408
583
22
621
984
791
331
499
759
212
877
100
967
10
455
29
707
955
943
564
923
90
52
261
321
838
691
327
980
961
212
985
412
834
797
762
644
391
771
217
639
808
466
909
546
215
880
226
870
169
53
925
471
150
970
405
292
454
214
243
429
861
536
227
847
779
349
709
766
324
242
763
816
61
376
297
51
893
736
88
845
546
364
661
861
830
584
727
955
478
357
5
293
323
854
784
3
500
610
561
46
653
430
428
95
566
818
313
636
663
280
296
92
692
721
869
980
658
52
566
842
667
355
92
100
347
323
24
757
466
150
279
655
666
483
39
571
101
716
993
5
671
914
185
822
416
311
86
127
235
241
177
820
513
882
539
983
418
25 563
94
717
257
202
508
984
122
788
39
964
104
183
280
429
369
595
163
313
829
448
12
287
145
666
222
18 1386
856
877
813
755
349
739
281
42
535
198
511
346
245
691
437
537
237
593
13 1034
748
685
237
735
82
247
524
45
277
437
709
730
519
9 3075
473
205
417
890
557
601
830
574
454
26 6271
260
304
503
983
866
276
348
679
714
295
282
654
248
176
232
128
224
779
195
728
913
859
307
399
757
589
9 60976905
971439797
4065127
290556595
301350292
587225441
888894937
693869803
90907495
687163272
33 1259
491
794
153
843
285
521
823
601
344
95
348
885
61
28
406
418
51
186
276
673
369
638
236
345
624
884
116
893
20
115
696
510
538
26 980
226
810
68
511
274
442
297
978
305
900
332
377
947
848
667
761
49
426
835
356
844
20
450
199
433
73
46 8365
247
148
979
94
882
152
587
540
807
827
795
718
943
117
474
124
799
558
123
738
370
478
921
142
806
126
863
208
179
806
379
678
383
485
211
715
463
667
695
926
670
205
452
884
302
446
41 2923
602
844
787
904
990
873
141
617
557
426
679
802
229
294
912
385
596
376
726
584
762
308
602
702
144
706
104
233
618
346
430
962
860
495
37
348
521
870
210
743
613
16 4320
157
655
681
659
290
969
962
469
149
162
359
552
352
297
704
304
34 975
44
407
15
561
957
601
110
613
548
186
269
696
448
842
190
897
803
152
806
230
194
397
128
114
481
155
822
131
108
320
531
781
466
846
24 999999999
543085755
360060386
501829784
97984070
753893208
171613838
75817746
945480838
687721667
367228239
847573195
405552761
445807759
873121248
51275165
435212925
250088699
691070561
548850879
760195893
861257322
83992575
646516503
956106917
34 780
68
861
480
878
788
237
779
464
114
816
900
251
548
475
855
599
609
235
94
672
691
794
186
18
121
910
66
832
117
90
649
12
114
417
37 1419
223
792
798
964
441
160
216
431
553
746
814
917
618
38
959
220
168
20
682
127
694
107
302
708
970
857
353
985
771
680
139
145
332
46
911
974
759
46 4272
76
199
519
443
937
270
653
298
540
963
119
312
48
947
377
911
405
526
480
223
554
654
344
206
380
394
93
472
883
739
601
210
469
90
309
683
186
640
295
156
683
991
983
385
625
692
42 809
126
654
610
216
270
81
421
237
920
572
41
845
969
875
434
229
392
408
10
335
732
451
172
260
722
140
706
693
984
642
744
538
437
183
919
888
15
157
387
858
837
361
44 1190
710
590
695
140
424
869
960
638
764
654
321
818
888
899
884
798
287
776
856
195
715
464
649
909
58
20
181
136
760
744
668
330
15
53
813
842
629
912
48
14
757
954
585
763
9 539
438
502
909
437
504
694
548
86
36
8 1000000000
555187286
483651468
898821572
247367982
369182944
601145651
827678518
840686907
29 329
322
160
201
495
277
393
56
57
296
132
68
14
258
630
12
989
866
716
411
228
6
431
923
44
373
926
223
584
354
17 2015
858
659
865
692
685
111
979
901
405
327
445
606
65
474
732
882
821
35 4287
395
935
647
615
929
66
598
652
786
199
708
211
502
582
300
660
618
323
432
793
998
666
970
759
699
838
114
988
727
879
650
64
445
152
274
10 3757
739
830
221
225
485
269
919
296
679
640
12 1679
132
428
221
630
640
350
712
422
467
102
80
270
11 437
216
554
164
467
864
281
518
403
198
23
141
12 999999999
616560409
508855970
770866744
675027905
975603126
946015948
759729209
547116283
184486859
254371558
622207633
611076879
23 43
571
650
795
364
650
648
218
588
711
1
395
108
346
758
214
409
62
79
594
408
694
543
578
16 289
856
634
973
186
129
868
550
638
87
923
634
81
522
293
10
187
37 1065
472
833
203
153
551
981
376
241
284
375
798
711
79
796
518
619
295
149
179
137
92
79
15
764
522
467
96
92
788
834
665
311
627
182
903
429
970
32 2012
681
448
138
592
793
530
322
706
564
306
36
96
740
695
496
681
33
737
957
954
484
373
380
873
409
483
853
303
931
551
467
713
10 1938
572
138
787
944
434
114
206
410
363
335
6 1142
127
910
590
324
577
960
37 294124671
7149027
923704687
683170828
693505288
511622442
923334770
104098416
152379042
675539364
847361509
46143825
754050523
482078464
826478935
418277118
251836703
814777706
466267052
359563620
339979158
276595685
489245426
684070674
594134399
55930945
669475368
223586504
4142601
475430310
121027230
278127101
399803434
153332035
565318391
908555531
248803455
831875939
46 978
327
359
392
397
481
843
449
209
864
694
585
975
94
236
207
372
802
39
575
340
673
891
841
347
929
908
151
869
153
71
308
594
573
641
246
199
139
829
241
614
703
746
864
216
11
755
37 994
427
620
345
431
769
235
610
593
698
975
14
161
570
144
220
738
77
669
213
483
359
620
689
89
803
110
430
582
336
431
349
87
200
909
826
648
883
43 580
375
486
433
720
852
962
799
861
142
51
949
752
830
782
660
643
261
902
656
235
301
631
199
454
967
862
403
683
119
83
333
890
7
452
806
468
959
80
136
142
536
782
834
26 2450
583
656
537
216
90
908
171
232
749
472
266
884
732
682
50
420
157
760
716
523
547
389
247
869
613
253
43 5062
819
421
750
222
594
61
840
677
598
184
840
69
753
857
261
638
701
959
622
326
771
290
975
439
535
153
299
189
464
933
103
816
671
500
243
552
474
623
292
334
63
748
337
9 240
231
92
730
781
118
325
311
16
34
22 194323230
647727732
263268217
481285983
549058506
349575892
398774093
580771435
308700428
410862765
295151226
163138703
4739591
233470809
147091175
674103827
743837593
743743901
355501632
652900313
861953687
881603659
635600630
10 425
247
324
730
536
31
721
26
25
988
488
29 2144
786
202
90
965
418
831
814
702
857
611
761
39
921
579
178
702
428
270
759
248
285
600
177
546
185
700
297
379
484
40 616
780
578
745
759
626
942
574
976
246
868
826
593
863
555
92
70
102
83
306
384
319
11
932
380
568
493
992
866
191
509
723
283
897
8
322
518
390
406
285
567
47 363
287
395
601
562
924
770
833
496
981
537
282
346
427
250
403
159
383
682
4
424
785
913
165
405
189
552
896
385
843
80
795
412
953
706
115
861
488
730
161
972
69
876
297
920
314
163
603
30 1311
323
815
23
114
777
544
453
756
948
835
571
823
221
930
867
961
376
728
626
729
797
682
634
319
578
708
684
269
31
942
38 656
211
349
654
633
199
704
443
509
862
564
505
960
456
309
633
403
148
823
561
111
274
920
101
451
97
203
319
948
157
9
66
554
108
187
658
690
148
398
10 1000000000
175370938
201837592
582618696
327878254
700705720
883036265
408986022
279820888
117589658
409193191
20 517
849
127
970
536
889
654
669
875
913
690
609
264
724
957
317
14
953
127
945
239
34 1300
74
858
915
539
779
920
421
90
869
878
734
904
59
489
720
616
477
173
939
336
526
20
294
700
440
162
107
316
67
681
526
965
831
95
15 1727
648
470
927
924
64
281
70
465
526
906
416
651
997
612
351
40 6776
529
878
809
178
734
364
207
176
221
329
399
228
173
741
832
920
500
652
294
270
728
306
104
638
88
302
321
91
854
453
649
742
511
301
951
265
281
535
362
145
10 2022
954
842
119
628
385
257
389
955
520
481
24 3600
828
196
807
789
174
821
421
417
448
94
115
143
564
558
468
492
80
749
827
497
450
477
753
541
29 999999999
633478321
348094326
211482622
242857872
260211488
808510929
50256131
907688501
693280943
761203851
733229606
75726351
740780374
928703460
148920588
974470194
266759747
729402833
401588590
904024041
836485885
572476324
543514950
541109806
332923909
697318014
931560825
459794818
426550609
48 1023
157
720
135
194
115
88
237
23
886
781
450
854
280
628
99
227
53
931
741
928
434
522
870
740
765
328
179
11
21
619
31
777
798
132
102
284
676
316
614
64
543
582
339
518
259
743
391
192
13 2276
243
131
99
358
597
745
937
483
657
815
709
445
160
19 2590
608
74
207
956
966
960
870
171
699
509
763
391
496
667
745
511
992
330
944
21 2027
982
52
104
288
447
61
161
940
56
61
296
652
592
260
369
324
187
987
347
266
605
18 3267
425
645
223
904
320
266
767
99
962
644
862
830
538
821
259
238
544
724
43 1576
565
34
19
471
333
488
355
983
785
943
34
530
587
54
643
733
288
132
863
825
530
929
685
149
383
881
622
969
392
671
405
91
685
460
401
583
677
654
233
696
746
829
303
42 734173389
502954520
810094714
72809368
9063869
206357649
310539208
395888713
883505278
511675860
873047754
887558672
907797159
972693791
895542968
999375616
813787503
249516818
533542795
927665710
807746921
320868034
799141721
332918727
13126905
738645001
231019230
968790460
511286923
108221662
325776390
358290987
925155708
523468421
113557212
37851818
993415743
970706518
385927383
40894101
150977080
50013416
956545530
26 244
442
928
960
546
29
291
5
454
640
272
748
759
543
35
322
124
394
345
190
417
118
285
995
334
307
740
19 525
209
232
824
15
898
968
606
888
730
493
521
87
442
431
882
402
849
380
67
30 683
495
350
883
710
12
444
304
937
312
855
811
686
812
811
49
679
559
238
624
94
484
573
816
282
917
991
743
431
224
859
48 186
436
740
317
522
970
22
218
595
823
321
583
639
456
154
763
670
643
2
544
880
362
339
737
49
488
190
599
235
787
918
871
498
592
484
436
201
438
611
238
529
488
991
418
331
608
412
33
814
47 1546
447
418
926
78
252
598
184
698
677
986
707
413
218
951
611
27
836
836
187
611
17
78
495
858
690
725
261
212
289
531
965
791
389
463
895
332
322
469
726
315
160
947
557
266
306
127
637
36 1104
251
681
789
267
320
441
984
964
28
527
284
58
917
158
419
693
381
306
554
565
584
241
399
611
359
40
644
987
437
909
16
997
480
600
690
167
43 158625449
40620059
488251251
280273056
474957040
732338522
711183780
217530242
1911150
392602931
818439430
409365264
885794329
613265078
868632690
521161436
959600686
163748450
549229187
69338008
50418385
871478950
874377822
494100705
507283651
691886378
250014625
744411742
689955712
576500409
634238704
178206637
18553120
270046040
166136888
814582264
743303201
439226422
839774069
150520991
770490849
887127184
978649863
746334376
34 1625
25
816
240
90
739
608
626
257
320
438
131
587
566
40
909
542
370
104
839
927
346
919
903
198
95
508
964
198
700
294
319
553
295
397
13 2023
992
995
152
934
751
440
679
112
126
709
88
309
503
7 154
14
893
823
255
846
827
409
27 407
632
712
529
8
163
158
601
883
173
57
473
182
828
364
818
772
486
416
726
219
639
409
442
461
859
114
901
12 1134
315
655
393
242
280
987
221
553
376
54
580
943
2 447
519
448
34 1000000000
719802998
477511388
138657324
712972700
708165166
438197502
22181875
154451025
659174662
614032879
77994938
113384889
967157195
951632308
856403628
367671353
294067712
681551770
529531318
157015423
695694350
709259971
136002852
477819917
649777627
122650316
105617836
386330694
385750795
15896112
841529750
798940542
656734069
264249413
15 377
31
14
990
492
286
947
190
374
407
969
137
399
535
748
339
1 689
689
26 4213
889
765
300
972
980
543
714
352
865
207
755
646
900
490
708
479
351
410
989
275
876
687
391
143
424
86
3 996
481
636
332
8 2326
890
194
589
366
702
407
710
179
44 510
827
572
114
129
538
352
566
426
242
547
557
541
787
625
410
443
861
278
884
789
534
28
438
333
986
251
910
506
987
314
288
274
66
448
6
325
128
597
374
275
258
440
687
391
8 85950071
360877600
412297516
6611544
200700393
797248414
253187978
883941911
717454939
18 2178
855
797
360
78
351
554
613
953
354
257
417
66
192
815
945
316
986
484
23 1074
259
684
317
422
896
25
355
284
573
640
449
701
206
701
466
587
534
308
792
756
421
339
671
10 2992
549
700
639
742
846
475
566
176
653
577
24 674
594
31
934
745
88
731
242
694
857
437
177
913
642
186
716
763
720
529
15
751
632
513
793
568
6 2466
667
335
623
977
311
274
43 1161
609
214
602
234
775
25
254
750
671
763
405
675
502
137
444
801
190
176
18
14
544
832
600
312
308
969
556
682
169
538
924
723
55
679
595
625
978
523
140
887
839
626
715
2 151232057
151232057
280391671
24 1079
813
194
82
601
805
228
915
885
886
860
136
419
715
179
190
783
597
128
186
206
772
24
660
610


# Output Format
For the \(i\)th test case, print "`Case #i:` " followed by the number of ants you need to move to get all of the ants to lie on the same line.
Case #1: 3
Case #2: 2
Case #3: 0

# Sample Explanation
In the first case, the \(4\) ants are all on the line \(y = x\), so no ants need to be moved. \(0\) is the only answer that will be accepted for this case.

In the second case, the \(4\) ants are at the vertices of a square, so every line contains at most \(2\) of the \(4\) ants. \(2\) ants need to be moved, so the answers \(2\), \(3\), and \(4\) will be accepted for this case.

The third case is depicted below. Ants \(2\), \(4\), \(5\), and \(7\) all lie on the line \(y = \frac{3}{2}x + 1\). Moving the other \(3\) ants is the optimal way to get all of the ants on a single line, so any answer between \(3\) and \(6\) inclusive will be accepted for this case.

{{PHOTO_ID:1528828545185163|WIDTH:400}}
    """
     },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>